In [1]:
#Import libraries:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV
import matplotlib.pylab as plt
import mpl_toolkits
# import pandas for data wrangling
import pandas as pd


# import numpy for Scientific computations
import numpy as np


# import machine learning libraries
import xgboost as xgb
from sklearn.metrics import accuracy_score



In [2]:
train = pd.read_csv("clear_data.csv")

In [3]:
from sklearn.metrics import mean_squared_error

target = "price"
columns = train.columns.tolist()
columns = [c for c in columns if c not in ["price"]]
#lin_model = LinearRegression()
# Fit the model to the training data.
#lin_model.fit(train[columns], train[target])

In [7]:
from sklearn.model_selection import GridSearchCV
params = {
    #'max_depth': [10, 9, 11]
     #         'learning_rate': 0.1,
      #        'num_samples_split': 200,
    #          'subsample': np.arange(0.5, 1.0, 0.1),
    #           'colsample_bytree': np.arange(0.5, 1.0, 0.1),
    #           'colsample_bylevel': np.arange(0.5, 1.0, 0.1),
    #           'n_estimators': range(100, 1000, 200),
    #          'num_class': [10]
    #'gamma':[i/10.0 for i in range(0,5)]
  #  'subsample':[0.75, 0.8, 0.85],
 #'colsample_bytree':[0.65, 0.7, 0.75]
 #'reg_alpha':[100, 110, 120, 130]
              }
xgbr = xgb.XGBRegressor(seed = 20, objective='reg:squarederror', learning_rate=0.1, gamma = 0, n_estimators=700, min_child_weight=2, max_depth=10)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring = 'neg_mean_squared_error', 
                   verbose=1,
                   error_score='raise',
                   n_jobs=2)

clf.fit(train[columns], train[target])
print("Best parameters:", clf.best_params_)
print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best parameters: {'reg_alpha': 110}
Lowest RMSE:  75289.06647478521


In [4]:
def detect_outliers_zscore(copy, label):
    thres = 4
    data = copy[label]
    mean = np.mean(data)
    std = np.std(data)
    for i in range(0, len(train)):
        if i in data.index:
            element = data[i]
            z_score = (element-mean)/std
            if (np.abs(z_score) > thres):
                copy.at[i, label] = mean

In [13]:
xgbr = xgb.XGBRegressor(seed = 0, reg_alpha=110, objective='reg:squarederror', learning_rate=0.01, gamma = 0, subsample=0.8, colsample_bytree = 0.7, n_estimators=7000, max_depth=10, min_child_weight=2)
#xgbr = xgb.XGBRegressor(seed=0)
#training = train.sample(frac=0.7, random_state=5)
#test = train.loc[~train.index.isin(training.index)]
#detect_outliers_zscore(training, 'latitude')
columns = train.columns.tolist()
columns = [c for c in columns if c not in ['price']]
xgbr.fit(train[columns], train[target])
#predictions = xgbr.predict(test[columns])

#lin_mse = mean_squared_error(predictions, test[target], squared=False)
#print("Computed error:", lin_mse)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=2, missing=nan, monotone_constraints=None,
             n_estimators=7000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [14]:
test = pd.read_csv('test_clear.csv')
predictions = xgbr.predict(test[columns])

In [15]:
predictions = pd.DataFrame(predictions)
predictions.to_csv('result_xgb.csv', header=['price'])

In [28]:
from sklearn.model_selection import train_test_split
X = train.drop('price', axis=1)
y = train['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [29]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)


In [6]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }

In [7]:
def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']), eval_metric="merror",
            early_stopping_rounds=10)
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation,verbose=False)
    

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }


In [8]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

SCORE:                                                 
0.00013896609227348526                                 
SCORE:                                                                                
0.00013896609227348526                                                                
SCORE:                                                                                
0.00013896609227348526                                                                
SCORE:                                                                                
0.00013896609227348526                                                                
SCORE:                                                                                
0.00013896609227348526                                                                
SCORE:                                                                                
0.00013896609227348526                                                                
SCORE:            

In [30]:
#clf=xgb.XGBClassifier(
 #               n_estimators =space['n_estimators'], max_depth = int(best_hyperparams['max_depth']), gamma = best_hyperparams['gamma'],
  #              reg_alpha = int(best_hyperparams['reg_alpha']),min_child_weight=int(best_hyperparams['min_child_weight']),
   #             colsample_bytree=int(best_hyperparams['colsample_bytree']), eval_metric="merror", red_alpha=best_hyperparams['reg_lambda'],
    #    )
clf=xgb.XGBClassifier(
        n_estimators =180, max_depth = 6, gamma = 7.3534981981,min_child_weight=9,
        colsample_bytree=0.5335078892, eval_metric="merror", red_alpha=0.2847415396,
)
y_train = le.fit_transform(train[target])
clf.fit(train[columns], y_train)

[22:33:22] WARNING: ../src/learner.cc:767: 
Parameters: { "red_alpha" } are not used.



In [24]:
test = pd.read_csv('test_clear.csv')
clf.predict(test)

ValueError: feature_names mismatch: ['Unnamed: 0', 'latitude', 'longitude', 'category', 'kupcha', 'rooms', 'repairs', 'hypothec', 'NormArea', 'NormCurrent', 'NormMax', 'year', 'month', 'day', '1-ci mikrorayon q.', '2-ci mikrorayon q.', '20 Yanvar m.', '20-ci sahə q.', '28 May m.', '28 May q.', '3-cü mikrorayon q.', '4-cü mikrorayon q.', '5-ci mikrorayon q.', '6-cı mikrorayon q.', '7-ci mikrorayon q.', '8 Noyabr m.', '8-ci kilometr q.', '8-ci mikrorayon q.', '9-cu mikrorayon q.', 'ASAN Xidmət №1', 'ASAN Xidmət №2', 'ASAN Xidmət №3', 'ASAN Xidmət №5', 'Abşeron r.', 'Avtovağzal m.', 'Axundov bağı', 'Ayna Sultanova heykəli', 'Azadlıq Prospekti m.', 'Azadlıq meydanı', 'Azneft meydanı', 'Azərbaycan Dillər Universiteti', 'Azərbaycan kinoteatrı', 'Azərbaycan turizm institutu', 'Ağ şəhər', 'Ağ şəhər q.', 'Badamdar q.', 'Bakmil m.', 'Bakı Asiya Universiteti', 'Bakı Dövlət Universiteti', 'Bakı Musiqi Akademiyası', 'Bakı Slavyan Universiteti', 'Bakıxanov q.', 'Bayıl parkı', 'Bayıl q.', 'Beşmərtəbə